In [ ]:
# Auto-reload changes in src while developing
%load_ext autoreload
%autoreload 2
%aimport src.gee_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import ee, geetools
ee.Initialize()

In [ ]:
import sys, os
# Resolve repo root robustly (works when running from notebooks/)
nb_dir = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == 'notebooks' else os.getcwd()
repo_root = nb_dir
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
from src.gee_utils import (
    upload_to_gee,
    check_asset_types,
    inspect_asset_metadata,
    count_assets_in_folder,
    create_gee_folder,
    delete_all_subfolders,
    make_assets_public,
    walk_assets,
    make_tree_public,
)

In [32]:
import os

# Local folder with GeoTIFFs
local_folder = "../data/OkavangoDelta_flooding-master/Rasters/Annual_inundation_maps"

# GEE asset folder
gee_asset_folder = "projects/ee-okavango/assets/Inman_Lyons/Annual_inundation_maps"

# List all .tif files
tif_files = [f for f in os.listdir(local_folder) if f.endswith(".tif")]

for tif in tif_files:
    local_path = os.path.join(local_folder, tif)
    upload_to_gee(local_path, gee_asset_folder, execute=False)


TypeError: upload_to_gee() got an unexpected keyword argument 'execute'

In [9]:
asset_list = [d['id'] for d in 
              ee.data.listAssets("projects/ee-okavango/assets/dsms")['assets']]

asset_list

['projects/ee-okavango/assets/dsms/34s6807960_20230709_dtm_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807965_20230709_ds2_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807965_20230709_dsm_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807965_20230709_dtm_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807970_20230709_ds2_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807970_20230709_dsm_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807970_20230709_dtm_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807975_20230622_ds2_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807975_20230622_dsm_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807975_20230622_dtm_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807980_20230622_ds2_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807980_20230622_dsm_1_100_egm08_cog',
 'projects/ee-okavango/assets/dsms/34s6807980_20230622_dtm_1_100_egm08_cog',

In [7]:
asset_path = 'projects/ee-okavango/assets/shapes/testing_grounds_20241127'
asset = ee.FeatureCollection(asset_path)

# Get information about the asset
info = asset.getInfo()

info

{'type': 'FeatureCollection',
 'columns': {'system:index': 'String'},
 'version': 1733849180036364,
 'id': 'projects/ee-okavango/assets/shapes/testing_grounds_20241127',
 'properties': {'system:asset_size': 7369},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[22.969779905690068, -19.103550567786034],
      [22.982804225398247, -19.33262114707843],
      [23.180895834770727, -19.336720261980165],
      [23.17084528571743, -19.2183413491786],
      [23.160818226000984, -19.099959096000124],
      [22.969779905690068, -19.103550567786034]]]},
   'id': '00000000000000000000',
   'properties': {}}]}

In [5]:
import ee

# Initialize the Earth Engine API
ee.Initialize()

# Define the polygon by loading the FeatureCollection
polygon_fc = ee.FeatureCollection("projects/ee-okavango/assets/shapes/testing_grounds_20241127").first()
polygon_geometry = polygon_fc.geometry()

polygon_geometry = ee.Geometry.Polygon(
        [[[21.716601878680233, -17.986800918676114],
          [21.716601878680233, -18.198273997193347],
          [21.988513499773983, -18.198273997193347],
          [21.988513499773983, -17.986800918676114]]])
# Define the folder containing the images
folder_path = "projects/ee-okavango/assets/dsms"

# List all assets in the folder
asset_list = ee.data.listAssets({'parent': folder_path})['assets']

# Function to check if an image is outside the polygon
def is_image_outside_polygon(image_path, polygon_geometry):
    try:
        image = ee.Image(image_path)
        image_geometry = image.geometry()
        return not polygon_geometry.contains(image_geometry, 1).getInfo()
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return False

# Loop through assets and delete images outside the polygon
for asset in asset_list:
    asset_path = asset['name']

    if is_image_outside_polygon(asset_path, polygon_geometry):
        print(f"Deleting image: {asset_path}")
        ee.data.deleteAsset(asset_path)
    else:
        print(f"Keeping image: {asset_path}")


Deleting image: projects/ee-okavango/assets/dsms/34s6407855_20230619_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6407860_20230619_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6407860_20230619_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6407860_20230619_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6407865_20230619_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6407865_20230619_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6407865_20230619_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6407870_20230619_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6407870_20230619_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6407870_20230619_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6407875_20230619_ds2_1_100_egm08_cog
Deleting image: proje

Deleting image: projects/ee-okavango/assets/dsms/34s6457810_20230621_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457815_20230621_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457815_20230621_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457815_20230621_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457820_20230621_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457820_20230621_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457820_20230621_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457825_20230621_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457825_20230621_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457825_20230621_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457830_20230621_ds2_1_100_egm08_cog
Deleting image: proje

Deleting image: projects/ee-okavango/assets/dsms/34s6457965_20230621_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457970_20230621_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457970_20230621_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457970_20230621_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457975_20230621_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457975_20230621_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457975_20230621_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457980_20230621_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457980_20230621_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457980_20230621_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6457985_20230621_ds2_1_100_egm08_cog
Deleting image: proje

Deleting image: projects/ee-okavango/assets/dsms/34s6507910_20230619_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6507915_20230621_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6507915_20230621_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6507915_20230621_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6507920_20230621_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6507920_20230621_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6507920_20230621_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6507925_20230621_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6507925_20230621_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6507925_20230621_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6507930_20230621_ds2_1_100_egm08_cog
Deleting image: proje

Deleting image: projects/ee-okavango/assets/dsms/34s6557850_20230702_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6557855_20230702_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6557855_20230702_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6557855_20230702_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6557860_20230625_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6557860_20230625_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6557860_20230625_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6557865_20230625_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6557865_20230625_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6557865_20230625_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6557870_20230625_ds2_1_100_egm08_cog
Deleting image: proje

...

Deleting image: projects/ee-okavango/assets/dsms/34s6707815_20230702_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707820_20230702_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707820_20230702_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707820_20230702_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707825_20230702_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707825_20230702_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707825_20230702_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707830_20230702_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707830_20230702_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707830_20230702_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707835_20230702_ds2_1_100_egm08_cog
Deleting image: proje

Deleting image: projects/ee-okavango/assets/dsms/34s6707975_20230622_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707980_20230622_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707980_20230622_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707980_20230622_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707985_20230622_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707985_20230622_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707985_20230622_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707990_20230701_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707990_20230701_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707990_20230701_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6707995_20230701_ds2_1_100_egm08_cog
Deleting image: proje

Deleting image: projects/ee-okavango/assets/dsms/34s6757900_20230625_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6757905_20230625_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6757905_20230625_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6757905_20230625_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6757910_20230625_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6757910_20230625_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6757910_20230625_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6757915_20230625_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6757915_20230625_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6757915_20230625_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6757920_20230625_ds2_1_100_egm08_cog
Deleting image: proje

Deleting image: projects/ee-okavango/assets/dsms/34s6807825_20230702_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6807830_20230702_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6807830_20230702_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6807830_20230702_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6807835_20230702_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6807835_20230702_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6807835_20230702_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6807840_20230702_ds2_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6807840_20230702_dsm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6807840_20230702_dtm_1_100_egm08_cog
Deleting image: projects/ee-okavango/assets/dsms/34s6807845_20230702_ds2_1_100_egm08_cog
Deleting image: proje

KeyboardInterrupt: 

In [ ]:

# def set_folder_permissions(folder_path, email, role="READER"):
#     # List all assets in the folder
#     asset_list = ee.data.listAssets({'parent': folder_path})['assets']
    

#     for asset in asset_list:
#         asset_id = asset['id']
#         # Get current access control list (ACL) for the asset
#         acl = ee.data.getAssetAcl(asset_id)
        
#         # Update ACL to share with the new user
#         acl['owners'] = acl.get('owners', [])
#         acl['writers'] = acl.get('writers', [])
#         acl['readers'] = acl.get('readers', [])
#         print (acl['readers'] )
#         if role == "READER":
#             acl['readers'].append(f"user:{email}")  # Add the user prefix
#         elif role == "WRITER":
#             acl['writers'].append(f"user:{email}")  # Add the user prefix
        
        
#         # Set the new ACL for the asset
#         ee.data.setAssetAcl(asset_id, acl)

# # Example usage
# folder_path = "projects/ee-quatratavia/assets/berms/DEP3"
# user_email = 'aaronkoop@gmail.com'  # Email of the user you want to share with
# set_folder_permissions(folder_path, user_email, role="READER")  # Role can be "READER" or "WRITER"


In [38]:
src = ee.Asset("projects/ee-quatratavia/assets/monthly_LST")
dst = ee.Asset("projects/ee-quatratavia/assets/CA_fire/monthly_LST")

src.move(dst)


ee.Asset('projects/ee-quatratavia/assets/CA_fire/monthly_LST')

In [39]:
src = ee.Asset("projects/ee-quatratavia/assets/MBTS")
dst = ee.Asset("projects/ee-quatratavia/assets/CA_fire/MBTS")

src.move(dst)


ee.Asset('projects/ee-quatratavia/assets/CA_fire/MBTS')

In [34]:
# src = ee.Asset("projects/ee-quatratavia/assets/Altar050919")
# src.move(dst)
# dst = ee.Asset("projects/ee-quatratavia/assets/berms/Altar050919")
# src.move(dst)
# ee.Initialize
# src.move(dst)
# history